In [2]:
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())
import os
import openai
openai.api_key = os.getenv("OPENA_AI_KEY")
from pathlib import Path
import pandas as pd

from llama_index.core.query_engine import PandasQueryEngine
from prompt import new_prompt, instruction_str, context

from RAG.agent.src.prompt import context
# from llama_parse import LlamaParse
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from note_engine import note_engine
from RAG.agent.src.load_index import canada_engine
from load_index import load_data_to_sql_db

ModuleNotFoundError: No module named 'llama_parse'

In [2]:
import os
# os.getcwd()
# # get the parent directory
# os.path.dirname(os.getcwd())
# # join the parent directory with the agent_data directory
# os.path.join(os.path.dirname(os.getcwd()), 'agent_data', 'Canada.pdf')


In [2]:
filepath = Path(os.path.join(os.path.dirname(os.getcwd()), 'agent_data', 
                 'WorldPopulation2023.csv'))
dbpath = str(Path(os.path.join(os.path.dirname(os.getcwd()), 'agent_data', 
                 'WorldPopulation2023.db')))
conn, engine, data, _ = load_data_to_sql_db(filepath, dbpath, "population", )

In [1]:
from llama_index.core import SimpleDirectoryReader
from llama_index.readers.file.tabular import CSVReader

population_filepath = Path(os.path.join(os.path.dirname(os.getcwd()),
                                    'agent_data', 'csv'))
population_docs = SimpleDirectoryReader(population_filepath).load_data()


NameError: name 'Path' is not defined

In [6]:
# population_query_engine = PandasQueryEngine(population, verbose=True)
# population_query_engine.update_prompts({"pandas_prompt": new_prompt})
# query_string = "What is the population of the Canada in 2023?"
# population_query_engine.query(query_string)

In [3]:
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding

model_name = "gpt-3.5-turbo"
embedding_model_name = "text-embedding-3-small"
llm = OpenAI(temperature=0.1, model=model_name)
embed_model = OpenAIEmbedding(model=embedding_model_name)
Settings.llm = llm
Settings.embed_model = embed_model
table_name = "population"

In [4]:
from llama_index.core.indices.struct_store import NLSQLTableQueryEngine
from load_index import text_to_query_engine

_, sql_database = text_to_query_engine(model_name, embedding_model_name, table_name, engine)

# Set up text2SQL prompt
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=[table_name],)


In [5]:
tools = [
    note_engine,
    QueryEngineTool(
        # query_engine=population_query_engine,
        query_engine=sql_query_engine,
        metadata=ToolMetadata(
            name="sql_on_population_data",
            description="this gives information at the world population and demographics",
        ),
    ),
    QueryEngineTool(
        query_engine=canada_engine,
        metadata=ToolMetadata(
            name="canada_data",
            description="this gives detailed information about canada the country",
        ),
    ),
]

llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = ReActAgent.from_tools(tools, llm=llm, verbose=True, context=context)

In [6]:
while (prompt := input("Enter a prompt (q to quit): ")) != "q":
    result = agent.query(prompt)
    print(result)

Thought: I need to use a tool to help me answer the question.
Action: sql_on_population_data
Action Input: {'input': "SELECT population FROM countries WHERE name='Nigeria' AND year=2023"}
Observation: The population of Nigeria in 2023 is projected to be 223,804,632.
Thought: I can answer without using any more tools.
Answer: The population of Nigeria in 2023 is projected to be 223,804,632.
The population of Nigeria in 2023 is projected to be 223,804,632.
Thought: I need to use a tool to help me answer the question.
Action: sql_on_population_data
Action Input: {'input': "SELECT population FROM countries WHERE country_name='Canada' AND year=2023"}
Observation: The population of Canada in 2023 is approximately 38,781,291.
Thought: I can answer without using any more tools.
Answer: The population of Canada in 2023 is approximately 38,781,291.
The population of Canada in 2023 is approximately 38,781,291.


In [8]:
canada_engine.query( "What is the capital of Canada?")

Response(response='Ottawa', source_nodes=[NodeWithScore(node=TextNode(id_='34f864c2-884c-460c-b3e0-22de75b220b3', embedding=None, metadata={'page_label': '1', 'file_name': '/Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/embeddings/RAG/agent/agent_data/Canada.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='c90580c2-8400-43d1-947c-f576a62210ad', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': '/Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/embeddings/RAG/agent/agent_data/Canada.pdf'}, hash='d34863186039bad427fea0e6c6413d9eac1111c84fe0da31e30977a6119ceacc'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='c85279e0-e779-407b-87fa-889877ea71f6', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='8f754dc255dbadd016499180497f3e293794df838a1440077df0bbab4014d413')}, text='Canada\nCountry in North America / Fr

In [10]:
# from llama_index.embeddings.openai import OpenAIEmbedding
# from llama_index.readers.file import PDFReader
# from llama_index.core import StorageContext, VectorStoreIndex, load_index_from_storage
# import chromadb
# from llama_index.vector_stores.chroma import ChromaVectorStore
# import os
# from pathlib import Path
# 
# 
# def get_index(data, index_name, embed_model, chromapath):
#     index = None
#     db = chromadb.PersistentClient(path=chromapath)
#     chroma_collection = db.get_or_create_collection(index_name)
#     vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
#     if not os.path.exists(chromapath):
#         print("building index", index_name)
#         storage_context = StorageContext.from_defaults(vector_store=vector_store)
#         index = VectorStoreIndex.from_documents(data, storage_context=storage_context, embed_model=embed_model,
#                                                 show_progress=True)
#     else:
#         index = VectorStoreIndex.from_vector_store(vector_store,
#                                                    embed_model=embed_model, )
# 
#     return index

In [12]:

# embedding_model_name = "text-embedding-3-large"
# embed_model = OpenAIEmbedding(model=embedding_model_name)
# 
# canada_filepath = Path.joinpath(Path.cwd().parent, 'agent_data', 'Canada.pdf')
# chromapath = str(
#     Path.joinpath(Path.cwd().parent, 'agent_data', 'agent_chroma_db'))
# canada_pdf = PDFReader().load_data(file=canada_filepath)
# canada_index = get_index(canada_pdf, "canada", embed_model,
#                          chromapath)
# canada_engine = canada_index.as_query_engine()

In [14]:
print(canada_engine.query("What is the captital of Canada"))

Ottawa
